In [7]:
import pandas as pd # for aristhmetic computing
import numpy as np 
import math
from sklearn import * # for ML
import datetime # for timing
import os,sys,gc # for debuging
import re, string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.lancaster import *
import numba
import time
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

DataBase = '../data'
DataSet = {
    'tra': pd.read_csv('%s/raw/train.csv' % DataBase),
    'tes': pd.read_csv('%s/raw/test.csv' % DataBase),
    'sub': pd.read_csv('%s/raw/sample_submission.csv' % DataBase)
}

stopwords_set = set(stopwords.words('english'))

#@numba.jit
def ApplyBOW(ColValues):
    ''''''
    n = len(ColValues)
    result = ['' for i in range(n)]
    for i in range(n):
        row = str(ColValues[i])
        # remove punctations
        tokenizer = RegexpTokenizer(r'\w+')
        if(row == np.nan):
            word_vec = []
        try:
            word_vec = tokenizer.tokenize(row)
        except Exception as e:
            print(type(row))
            sys.exit(1)
        # remove stopwords
        word_vec = [word for word in word_vec if(word not in stopwords_set)]
        word_vec = [word.lower() for word in word_vec]
        # stem words
        #lanste = LancasterStemmer()
        #word_vec = [lanste.stem(i) for i in word_vec]
        result[i] = ' '.join(word_vec)
    return result

for mod in ['tra', 'tes']:
    s = time.time()
    #ret = ApplyBOW(DataSet[mod]['comment_text'][:10].values) 
    DataSet[mod]['comment_text'] = ApplyBOW(DataSet[mod]['comment_text'].values) 
    e = time.time()
    print('time %.2fs' % ((e - s)))
    #

AssertionError: Failed at object (analyzing bytecode)
SETUP_EXCEPT(arg=14, lineno=35)

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

# def get_wordnet_pos(treebank_tag):

#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
#     elif treebank_tag.startswith('N'):
#         return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return ''
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('fuck', wordnet.NOUN)

'fuck'

In [17]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
## resampling bootstrap
R = 3 # bootstrap number
RRate = 0.9 # resampling rate
HRate = 0.1 # holdout rate
K = 5 # folds number
for i in range(R):
    BoostrapOutputDir = '%s/bootstrap/%s' % (DataBase, i)
    ## sampling
    ReSampledTrainData = TrainData.sample(frac= RRate, random_state= 2017 + i)
    print('sampled data rate %s/%s' % (len(ReSampledTrainData), len(TrainData)))
    ## leave-one-out(holdout) strategy for local validation
    np.random.seed(2017 + i)
    msk = np.random.rand(len(ReSampledTrainData)) < HRate
    holdout = ReSampledTrainData[msk]
    train = ReSampledTrainData[~msk]
    print('LOO data rate %s/%s' % (len(holdout), len(ReSampledTrainData)))
    ## CV before engineering in case of data-leak for feature/model selection
    kf = model_selection.KFold(n_splits = K, random_state = 2017 + i, shuffle = True)
    OutputDir = '%s/l0' % BoostrapOutputDir
    if(os.path.exists('%s/kfold' % OutputDir) == False):
        os.makedirs('%s/kfold' % OutputDir)
    for fold, (train_index, test_index) in enumerate(kf.split(train)):
        FoldTrain, FoldValid = train.iloc[train_index].copy(), train.iloc[test_index].copy()
        FoldHoldout = holdout.copy()
        FoldTest = TestData.copy()
        print('valid data rate %s/%s' % (len(FoldValid), len(train)))
        ## TODO
        # log1p length
        FoldTrain['total_log1p_len'] = np.log1p(FoldTrain['comment_text'].str.len())
        # none label
        FoldTrain['none'] = 1 - FoldTrain[label_cols].max(axis=1)
        # fill null comment text
        FoldTrain[COMMENT].fillna("unknown", inplace= True)
        # 
        ## save
        FoldOutputDir = '%s/kfold/%s' % (OutputDir, fold)
        if(os.path.exists(FoldOutputDir) == False):
            os.makedirs(FoldOutputDir)
        FoldValid.to_csv('%s/valid.csv' % FoldOutputDir, index= False)
        #FoldTest.to_csv('%s/holdout.csv' % FoldOutputDir, index= False)
        #FoldTest.to_csv('%s/test.csv' % FoldOutputDir, index= False)
        print('fold %s done.' % fold)
    print('boostrap %s done' % i)
    break ## for 

sampled data size 86266
LOO size 8715
fold 0 done.
fold 1 done.
fold 2 done.
fold 3 done.
fold 4 done.
boostrap 0 done
sampled data size 86266
LOO size 8574
fold 0 done.
fold 1 done.
fold 2 done.
fold 3 done.
fold 4 done.
boostrap 1 done
sampled data size 86266
LOO size 8679
fold 0 done.
fold 1 done.
fold 2 done.
fold 3 done.
fold 4 done.
boostrap 2 done
